In [10]:
import pandas as pd
import numpy as np

df = pd.read_csv("C:/Users/Sean/Desktop/UoPX/tv_unit.csv")

# Assume you've already read your CSV into a DataFrame `df`
# e.g., df = pd.read_csv("path/to/your_data.csv")

# Group by `ad_length` and calculate the mean of `convert`
conversion_by_length = df.groupby('ad_length')['convert'].mean()

print(conversion_by_length)


ad_length
15    0.411765
30    0.600000
Name: convert, dtype: float64


In [11]:
conversion_rate_15 = df[df['ad_length'] == 15]['convert'].mean()
conversion_rate_30 = df[df['ad_length'] == 30]['convert'].mean()

print("Conversion rate for 15-second ads:", conversion_rate_15)
print("Conversion rate for 30-second ads:", conversion_rate_30)


('Conversion rate for 15-second ads:', 0.41176470588235292)
('Conversion rate for 30-second ads:', 0.59999999999999998)


In [12]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [13]:
conditions = [
    df['ad_length'] == 15,
    df['ad_length'] == 30
]
choices = [1, 2]

# Use np.select to create the ad_signal column. 
# For ad_length values that don't match, it returns np.nan (you can change this default if desired).
df['ad_signal'] = np.select(conditions, choices, default=np.nan)

print(df)

   ip_address    ad_date ad_name  ad_length  convert convert_date  ad_signal
0         ip1   1/1/2023     ad1         15        0          NaN        1.0
1         ip1   2/6/2023     ad2         15        0          NaN        1.0
2         ip1   3/9/2023     ad3         15        0          NaN        1.0
3         ip1   7/6/2023     ad1         30        0          NaN        2.0
4         ip1   8/4/2023     ad4         15        0          NaN        1.0
5         ip2   1/1/2023     ad1         15        1   10/10/2023        1.0
6         ip2   3/6/2023     ad2         15        1   10/10/2023        1.0
7         ip2   3/9/2023     ad3         15        1   10/10/2023        1.0
8         ip2   7/6/2023     ad1         30        1   10/10/2023        2.0
9         ip2   8/4/2023     ad4         15        1   10/10/2023        1.0
10        ip2   8/7/2023     ad2         15        1   10/10/2023        1.0
11        ip2   9/1/2023     ad3         30        1   10/10/2023        2.0

In [6]:
from scipy.stats import chi2_contingency

# Build a contingency table for convert (0 or 1) vs. ad_length=30 or not
df['is_30sec'] = (df['ad_length'] == 30).astype(int)

contingency_table = pd.crosstab(df['is_30sec'], df['convert'])
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-square:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies:\n", expected)

('Chi-square:', 0.053921568627450893)
('p-value:', 0.81637476978602408)
('Degrees of freedom:', 1L)
('Expected frequencies:\n', array([[ 9.27272727,  7.72727273],
       [ 2.72727273,  2.27272727]]))


In [15]:
# adstocking of column ad_length


# Convert the ad_date column to datetime.
df['ad_date'] = pd.to_datetime(df['ad_date'], format='%m/%d/%Y')

# Define the decay rate (lambda) per day.
decay_rate = 0.1

def compute_adstock(group, decay_rate=decay_rate):
    # Sort the group by ad_date.
    group = group.sort_values('ad_date').copy()
    
    adstock_values = []
    previous_adstock = 0
    previous_date = None
    
    # Iterate through the rows in the sorted group.
    for _, row in group.iterrows():
        # For the first row, no prior ad exists.
        if previous_date is None:
            effective_decay = 1  # No decay applied.
        else:
            # Calculate the time difference in days.
            delta_days = (row['ad_date'] - previous_date).days
            # Calculate effective decay factor based on the time difference.
            effective_decay = np.exp(-decay_rate * delta_days)
        
        # Compute current adstock.
        current_adstock = row['ad_signal'] + effective_decay * previous_adstock
        
        # Append the computed adstock.
        adstock_values.append(current_adstock)
        
        # Update previous values.
        previous_adstock = current_adstock
        previous_date = row['ad_date']
    
    # Add the computed adstock values as a new column.
    group['adstock'] = adstock_values
    return group

# Apply the adstock calculation for each ip_address group.
df = df.groupby('ip_address', group_keys=False).apply(compute_adstock)

# Display the DataFrame with the new 'adstock' column.
print(df)


   ip_address    ad_date ad_name  ad_length  convert convert_date  ad_signal  \
0         ip1 2023-01-01     ad1         15        0          NaN        1.0   
1         ip1 2023-02-06     ad2         15        0          NaN        1.0   
2         ip1 2023-03-09     ad3         15        0          NaN        1.0   
3         ip1 2023-07-06     ad1         30        0          NaN        2.0   
4         ip1 2023-08-04     ad4         15        0          NaN        1.0   
5         ip2 2023-01-01     ad1         15        1   10/10/2023        1.0   
6         ip2 2023-03-06     ad2         15        1   10/10/2023        1.0   
7         ip2 2023-03-09     ad3         15        1   10/10/2023        1.0   
8         ip2 2023-07-06     ad1         30        1   10/10/2023        2.0   
9         ip2 2023-08-04     ad4         15        1   10/10/2023        1.0   
10        ip2 2023-08-07     ad2         15        1   10/10/2023        1.0   
11        ip2 2023-09-01     ad3        